In [4]:
# imports
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime, timedelta

In [5]:
vehicles = pd.read_csv('./data/Motor_Vehicle_Collisions_-_Vehicles_20241103.csv', index_col=0, low_memory=False)
# vehicles

In [7]:
# set(vehicles["CONTRIBUTING_FACTOR_1"])
print(vehicles.head)

<bound method NDFrame.head of            COLLISION_ID  CRASH_DATE CRASH_TIME  \
UNIQUE_ID                                        
10385780         100201  09/07/2012       9:03   
19140702        4213082  09/23/2019       8:15   
14887647        3307608  10/02/2015      17:18   
14889754        3308693  10/04/2015      20:34   
14400270         297666  04/25/2013      21:15   
...                 ...         ...        ...   
20774561        4767779  10/27/2024      15:16   
20774656        4767855  10/28/2024       7:45   
20773672        4767169  10/29/2024       4:25   
20774412        4767800  10/24/2024      14:42   
20774394        4767691  10/28/2024       8:37   

                                     VEHICLE_ID STATE_REGISTRATION  \
UNIQUE_ID                                                            
10385780                                      1                 NY   
19140702   0553ab4d-9500-4cba-8d98-f4d7f89d5856                 NY   
14887647                               

In [10]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all rows
print(vehicles.iloc[0])

COLLISION_ID                              100201
CRASH_DATE                            09/07/2012
CRASH_TIME                                  9:03
VEHICLE_ID                                     1
STATE_REGISTRATION                            NY
VEHICLE_TYPE                   PASSENGER VEHICLE
VEHICLE_MAKE                                 NaN
VEHICLE_MODEL                                NaN
VEHICLE_YEAR                                 NaN
TRAVEL_DIRECTION                             NaN
VEHICLE_OCCUPANTS                            NaN
DRIVER_SEX                                   NaN
DRIVER_LICENSE_STATUS                        NaN
DRIVER_LICENSE_JURISDICTION                  NaN
PRE_CRASH                                    NaN
POINT_OF_IMPACT                              NaN
VEHICLE_DAMAGE                               NaN
VEHICLE_DAMAGE_1                             NaN
VEHICLE_DAMAGE_2                             NaN
VEHICLE_DAMAGE_3                             NaN
PUBLIC_PROPERTY_DAMA

In [12]:
# filter by date 
cutoff_date = datetime.now() - timedelta(days=10*365)
vehicles['CRASH_DATE'] = pd.to_datetime(vehicles['CRASH_DATE'], format='%m/%d/%Y')
filtered_vehicles = vehicles[vehicles['CRASH_DATE'] >= cutoff_date]
print(filtered_vehicles.size)
print(vehicles.size)

79572432
102665184


In [14]:
print(filtered_vehicles.iloc[0])

COLLISION_ID                                                4213082
CRASH_DATE                                      2019-09-23 00:00:00
CRASH_TIME                                                     8:15
VEHICLE_ID                     0553ab4d-9500-4cba-8d98-f4d7f89d5856
STATE_REGISTRATION                                               NY
VEHICLE_TYPE                    Station Wagon/Sport Utility Vehicle
VEHICLE_MAKE                                          TOYT -CAR/SUV
VEHICLE_MODEL                                                   NaN
VEHICLE_YEAR                                                 2002.0
TRAVEL_DIRECTION                                              North
VEHICLE_OCCUPANTS                                               1.0
DRIVER_SEX                                                        M
DRIVER_LICENSE_STATUS                                      Licensed
DRIVER_LICENSE_JURISDICTION                                      NY
PRE_CRASH                                      G

In [ ]:
copy_vehicles = filtered_vehicles.copy()
copy_vehicles

In [ ]:
copy_vehicles.columns

#### ** TODO: **
Clean up columns
* Separate CRASH_DATE into 3 date fields, year, month day ✔
* Find out if date is a holiday?
* TRAVEL_DIRECTION ✔
* CONTRIBUTING_FACTOR_1, 2
* STATE_REGISTRATION ✔
* VEHICLE_DAMAGE, (20 features), maybe add  _1, _2, _3

In [ ]:
copy_vehicles["TRAVEL_DIRECTION"] = copy_vehicles["TRAVEL_DIRECTION"].replace('E', "East").replace('N', "North").replace('S', 'South').replace('W', 'West').replace('U', 'Unknown').replace('-', 'Unknown')
copy_vehicles["TRAVEL_DIRECTION"]= copy_vehicles["TRAVEL_DIRECTION"].fillna('Unknown')

In [ ]:
# set(copy_vehicles["TRAVEL_DIRECTION"])

In [ ]:
for col in copy_vehicles.columns:
    print(col)
    print(copy_vehicles[col].unique())

In [ ]:
if 'TRAVEL_DIRECTION' in copy_vehicles.columns:
    encoder = OneHotEncoder(sparse_output=False)
    encoded_data = encoder.fit_transform(copy_vehicles[["TRAVEL_DIRECTION"]])
    encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(["TRAVEL_DIRECTION"]))
    copy_vehicles = pd.concat([copy_vehicles.drop(columns=["TRAVEL_DIRECTION"]).reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)


In [ ]:
if 'STATE_REGISTRATION' in copy_vehicles.columns:
    encoder = OneHotEncoder(sparse_output=False)
    encoded_data = encoder.fit_transform(copy_vehicles[["STATE_REGISTRATION"]])
    encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(["STATE_REGISTRATION"]))
    copy_vehicles = pd.concat([copy_vehicles.drop(columns=["STATE_REGISTRATION"]).reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

In [ ]:
if 'CONTRIBUTING_FACTOR_1' in copy_vehicles.columns:
    encoder = OneHotEncoder(sparse_output=False)
    encoded_data = encoder.fit_transform(copy_vehicles[["CONTRIBUTING_FACTOR_1"]])
    encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(["CONTRIBUTING_FACTOR_1"]))
    copy_vehicles = pd.concat([copy_vehicles.drop(columns=["CONTRIBUTING_FACTOR_1"]).reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

In [ ]:
if 'VEHICLE_DAMAGE' in copy_vehicles.columns:
    encoder = OneHotEncoder(sparse_output=False)
    encoded_data = encoder.fit_transform(copy_vehicles[["VEHICLE_DAMAGE"]])
    encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(["VEHICLE_DAMAGE"]))
    copy_vehicles = pd.concat([copy_vehicles.drop(columns=["VEHICLE_DAMAGE"]).reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

In [ ]:
# DROP VEHICLE_ID
copy_vehicles

In [ ]:
# Add dates to separate fields
copy_vehicles["CRASH_DATE"] = pd.to_datetime(copy_vehicles["CRASH_DATE"], format='%m/%d/%Y')
copy_vehicles["CRASH_DATE_YEAR"] = copy_vehicles["CRASH_DATE"].dt.year
copy_vehicles["CRASH_DATE_MONTH"] = copy_vehicles["CRASH_DATE"].dt.month
copy_vehicles["CRASH_DATE_DAY"] = copy_vehicles["CRASH_DATE"].dt.day
copy_vehicles

In [ ]:
# Drop bad columns
if 'STATE_REGISTRATION_nan' in copy_vehicles.columns:
    copy_vehicles = copy_vehicles.drop(columns=["STATE_REGISTRATION_nan"])
if 'CONTRIBUTING_FACTOR_1_nan' in copy_vehicles.columns:
    copy_vehicles = copy_vehicles.drop(columns=["CONTRIBUTING_FACTOR_1_nan"])
if 'VEHICLE_DAMAGE_nan' in copy_vehicles.columns:
    copy_vehicles = copy_vehicles.drop(columns=["VEHICLE_DAMAGE_nan"])

# remove? CONTRIBUTING_FACTOR_1_1, CONTRIBUTING_FACTOR_1_80

In [ ]:
set(copy_vehicles.columns)

In [ ]:
set(copy_vehicles["VEHICLE_OCCUPANTS"])

In [ ]:
print(copy_vehicles.iloc[0])

TODO
* Drop vehicle year > year +2
* Drop vehicles older than 1970
* Determine vehicle type from description fields
* Cast VEHICLE_OCCUPANTS to int
* Drop invalid options (state=ZZ)